# ETL

This notebook is to extract from sqlite and mongodb. Transforms data and load/save into a dataframe

### a) Gather Data from sqlite.db

Extracting data from sqlite.db and transforming to get video duration, count, like, comments and get total of engagement

In [1]:
import os
import sqlite3
import pandas as pd

# setting path to database
folder_path = "data"
db_file_path = os.path.join(folder_path, "tiktok.db")


In [2]:
# connecting to db
conn = sqlite3.connect(db_file_path)

In [3]:
# SQL query to join VideoMetrics and Videos tables on video_id
query = '''
SELECT v.video_duration_sec AS tiktok_duration_sec,
        m.video_view_count AS tiktok_view_count,
        m.video_like_count AS tiktok_like_count,
        m.video_comment_count AS tiktok_comment_count
FROM Videos v
JOIN VideoMetrics m
ON v.video_id = m.video_id
WHERE v.video_duration_sec IS NOT NULL AND
        v.video_duration_sec > 0;
'''

In [4]:
# Load into df
tiktok_df = pd.read_sql_query(query, conn)

# close connection
conn.close()


In [5]:
tiktok_df.head()

,tiktok_duration_sec,tiktok_view_count,tiktok_like_count,tiktok_comment_count
0,59,343296.0,19425.0,0.0
1,32,140877.0,77355.0,684.0
2,31,902185.0,97690.0,329.0
3,25,437506.0,239954.0,584.0
4,19,56167.0,34987.0,152.0


In [6]:
# calculate total engagement by adding views, likes, and comments
tiktok_df['tiktok_total_engagement'] = (
    tiktok_df['tiktok_view_count'] +
    tiktok_df['tiktok_like_count'] +
    tiktok_df['tiktok_comment_count']
)

In [7]:
tiktok_df.head()

,tiktok_duration_sec,tiktok_view_count,tiktok_like_count,tiktok_comment_count,tiktok_total_engagement
0,59,343296.0,19425.0,0.0,362721.0
1,32,140877.0,77355.0,684.0,218916.0
2,31,902185.0,97690.0,329.0,1000204.0
3,25,437506.0,239954.0,584.0,678044.0
4,19,56167.0,34987.0,152.0,91306.0


In [8]:
tiktok_df.describe()

,tiktok_duration_sec,tiktok_view_count,tiktok_like_count,tiktok_comment_count,tiktok_total_engagement
count,19382.000000,19084.000000,19084.000000,19084.000000,1.908400e+04
mean,32.421732,254708.558688,84304.636030,349.312146,3.393625e+05
std,16.229967,322893.280814,133420.546814,799.638865,4.379451e+05
min,5.000000,20.000000,0.000000,0.000000,2.300000e+01
25%,18.000000,4942.500000,810.750000,1.000000,6.013750e+03
50%,32.000000,9954.500000,3403.500000,9.000000,1.376100e+04
75%,47.000000,504327.000000,125020.000000,292.000000,6.608205e+05
max,60.000000,999817.000000,657830.000000,9599.000000,1.656099e+06


In [9]:
missing_values_count = tiktok_df.isnull().sum()
missing_values_count

tiktok_duration_sec          0
tiktok_view_count          298
tiktok_like_count          298
tiktok_comment_count       298
tiktok_total_engagement    298
dtype: int64

In [10]:
tiktok_df = tiktok_df.dropna(subset=['tiktok_view_count'])

In [11]:
missing_values_count = tiktok_df.isnull().sum()
missing_values_count

tiktok_duration_sec        0
tiktok_view_count          0
tiktok_like_count          0
tiktok_comment_count       0
tiktok_total_engagement    0
dtype: int64

In [12]:
# saving data to pickle
tiktok_df.to_pickle("tiktok.pkl")

### b) Gather Data from mongoDB

Gathering data from mongoDB views, likes, comments, and duration

In [13]:
import pymongo

# Create the client
client = pymongo.MongoClient('localhost', 27017)

# Connect to our database
db = client['local']
collection = db["youtube_videos"]

In [14]:
cursor = collection.find()

# iterating values in document for views, like, comments, and duration
for document in cursor:
    view_count = document["view_count"]
    like_count = document["like_count"]
    comment_count = document["comment_count"]
    duration = document["duration"]
    print(f"View Count: {view_count}, Like Count: {like_count}, Comment Count: {comment_count}, Duration: {duration}")

View Count: 295836, Like Count: 4617, Comment Count: 3632, Duration: PT1M21S
View Count: 195318, Like Count: 988, Comment Count: 1868, Duration: PT1M19S
View Count: 172917, Like Count: 1389, Comment Count: 4055, Duration: PT2M15S
View Count: 88721, Like Count: 1095, Comment Count: 1833, Duration: PT5M32S
View Count: 513072, Like Count: 9989, Comment Count: 5751, Duration: PT50S
View Count: 466054, Like Count: 10013, Comment Count: 6296, Duration: PT6M5S
View Count: 268171, Like Count: 5498, Comment Count: 0, Duration: P0D
View Count: 4751, Like Count: 54, Comment Count: 117, Duration: PT4M12S
View Count: 14955, Like Count: 175, Comment Count: 505, Duration: PT3M7S
View Count: 38707, Like Count: 399, Comment Count: 227, Duration: PT7M55S
View Count: 153969, Like Count: 2274, Comment Count: 1688, Duration: PT5M19S
View Count: 40810, Like Count: 592, Comment Count: 0, Duration: P0D
View Count: 2417613, Like Count: 7366, Comment Count: 663, Duration: PT9H49M30S
View Count: 48016, Like Coun

In [15]:
# loading into a dataframe
data = []
cursor = collection.find()
for document in cursor:
    data.append({
        "youtube_view_count": document["view_count"],
        "youtube_like_count": document["like_count"],
        "youtube_comment_count": document["comment_count"],
        "youtube_duration": document["duration"]
    })

youtube_df = pd.DataFrame(data)

In [16]:
youtube_df.head()

,youtube_view_count,youtube_like_count,youtube_comment_count,youtube_duration
0,295836,4617,3632,PT1M21S
1,195318,988,1868,PT1M19S
2,172917,1389,4055,PT2M15S
3,88721,1095,1833,PT5M32S
4,513072,9989,5751,PT50S


In [17]:
youtube_df.dtypes

youtube_view_count        int64
youtube_like_count        int64
youtube_comment_count     int64
youtube_duration         object
dtype: object

In [18]:
import re

def iso8601_to_seconds(duration_str):
    """Converts an ISO 8601 duration string to seconds.

    Args:
        duration_str: The ISO 8601 duration string, e.g., "PT1H23M45S".

    Returns:
        The duration in seconds.
    """

    regex = r"PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?"
    match = re.match(regex, duration_str)

    if match:
        hours, minutes, seconds = match.groups()
        hours = int(hours) if hours else 0
        minutes = int(minutes) if minutes else 0
        seconds = int(seconds) if seconds else 0

        return hours * 3600 + minutes * 60 + seconds
    

In [19]:
# make new column into secs
youtube_df["youtube_duration_sec"] = youtube_df["youtube_duration"].astype(str).apply(iso8601_to_seconds)

In [20]:
youtube_df.head()

,youtube_view_count,youtube_like_count,youtube_comment_count,youtube_duration,youtube_duration_sec
0,295836,4617,3632,PT1M21S,81.0
1,195318,988,1868,PT1M19S,79.0
2,172917,1389,4055,PT2M15S,135.0
3,88721,1095,1833,PT5M32S,332.0
4,513072,9989,5751,PT50S,50.0


In [21]:
# to see how see if any rows have zero seconds or no value
zero_duration_rows = youtube_df[youtube_df['youtube_duration_sec'] == 0]
missing_values_count = youtube_df.isnull().sum()

In [22]:
zero_duration_rows

,youtube_view_count,youtube_like_count,youtube_comment_count,youtube_duration,youtube_duration_sec


In [23]:
missing_values_count

youtube_view_count        0
youtube_like_count        0
youtube_comment_count     0
youtube_duration          0
youtube_duration_sec     51
dtype: int64

In [24]:
# drop where null values for secs
youtube_df = youtube_df.dropna(subset=['youtube_duration_sec'])

# drop the 'youtube_duration' column
youtube_df = youtube_df.drop('youtube_duration', axis=1)


In [25]:
missing_values_count = youtube_df.isnull().sum()
missing_values_count

youtube_view_count       0
youtube_like_count       0
youtube_comment_count    0
youtube_duration_sec     0
dtype: int64

In [26]:
youtube_df.head()

,youtube_view_count,youtube_like_count,youtube_comment_count,youtube_duration_sec
0,295836,4617,3632,81.0
1,195318,988,1868,79.0
2,172917,1389,4055,135.0
3,88721,1095,1833,332.0
4,513072,9989,5751,50.0


In [27]:
# calculate total engagement by adding views, likes, and comments
youtube_df['youtube_total_engagement'] = (
    youtube_df['youtube_view_count'] +
    youtube_df['youtube_like_count'] +
    youtube_df['youtube_comment_count']
)

In [28]:
youtube_df.head()

,youtube_view_count,youtube_like_count,youtube_comment_count,youtube_duration_sec,youtube_total_engagement
0,295836,4617,3632,81.0,304085
1,195318,988,1868,79.0,198174
2,172917,1389,4055,135.0,178361
3,88721,1095,1833,332.0,91649
4,513072,9989,5751,50.0,528812


In [29]:
youtube_df.describe()

,youtube_view_count,youtube_like_count,youtube_comment_count,youtube_duration_sec,youtube_total_engagement
count,9.490000e+02,9.490000e+02,949.000000,949.000000,9.490000e+02
mean,1.075267e+07,9.620427e+04,3852.589041,1862.448894,1.085273e+07
std,1.978882e+08,3.375386e+05,20670.331892,6162.048262,1.979054e+08
min,0.000000e+00,0.000000e+00,0.000000,8.000000,0.000000e+00
25%,4.330600e+04,6.470000e+02,74.000000,79.000000,4.477900e+04
50%,3.001890e+05,4.421000e+03,524.000000,286.000000,3.089620e+05
75%,1.840992e+06,3.520300e+04,2213.000000,809.000000,1.887775e+06
max,6.082129e+09,4.336515e+06,526376.000000,42901.000000,6.082129e+09


In [31]:
youtube_df.to_pickle("youtube.pkl")

Data extracted from databases, transformed into suitable formats, and loading/save into dataframes